## Kvantna Furijeova Transformacija (QFT/KFT)

Kvantna Furijeova Transformacija (QFT/KFT) je ključna komponenta mnogih kvantnih algoritama. Ona transformiše bazno stanje u superpoziciju svih mogućih stanja, pri čemu svako dobija različitu fazu. Ovo je čini moćnim alatom za otkrivanje skrivenih obrazaca, periodičnih ponavljanja ili nepoznatih faza. Standardna implementacija QFT-a zasniva se na Koppersmitovoj formuli [1]:

$$
|x_{n-1}\dots x_0\rangle \;\mapsto\;
\frac{1}{\sqrt{2}}\Big(|0\rangle + e^{2\pi i \, 0.x_0}|1\rangle\Big)\;\otimes\;
\frac{1}{\sqrt{2}}\Big(|0\rangle + e^{2\pi i \, 0.x_1x_0}|1\rangle\Big)
\;\otimes\;\cdots\;\otimes\;
\frac{1}{\sqrt{2}}\Big(|0\rangle + e^{2\pi i \, 0.x_{n-1}x_{n-2}\dots x_0}|1\rangle\Big).
$$

gde $0.x_j x_{j-1} \dots x_0$ predstavlja binarnu razlomljenu vrednost. Na primer:

$$
0.x_2 x_1 x_0 = \frac{x_2}{2} + \frac{x_1}{4} + \frac{x_0}{8}.
$$

Ovde prvi kubit zavisi od **najmanje značajnog bita** $x_{0}$, a svaki naredni kubit akumulira sve više bitova s desna ka levo.

Međutim, u literaturi iz oblasti kvantnog računanja češće se koristi tumačenje iz udžbenika Nilsen i Čuang [2]. U ovom pristupu, prvi kubit zavisi od **najznačajnijeg bita** $x_{n-1}$, a svaki naredni kubit akumulira sve više bitova sleva ka desno:

$$
|x_{n-1}\dots x_0\rangle \;\mapsto\;
\frac{1}{\sqrt{2}}\Big(|0\rangle + e^{2\pi i \, 0.x_{n-1}}|1\rangle\Big)\;\otimes\;
\frac{1}{\sqrt{2}}\Big(|0\rangle + e^{2\pi i \, 0.x_{n-2}x_{n-1}}|1\rangle\Big)
\;\otimes\;\cdots\;\otimes\;
\frac{1}{\sqrt{2}}\Big(|0\rangle + e^{2\pi i \, 0.x_0x_1\dots x_{n-1}}|1\rangle\Big).
$$

Svaki od ovih faktora predstavlja jedan kubit pripremljen u superpoziciji oblika  
$\frac{1}{\sqrt{2}}\left(|0\rangle + e^{i\varphi}|1\rangle\right)$.  
Relativna faza $e^{i\varphi}$ koja se odnosi na komponentu $|1\rangle$ određena je binarnom razlomljenom vrednošću. Krećući se od najznačajnijeg ka najmanje značajnom kubitu (u Nilsen & Čuang konvenciji), svaki naredni kubit dobija fazni pomak sve finije rezolucije, u koracima od $1/2^k$. Zato su članovi zapisani redom od najvećeg ka najmanjem doprinosu faze.

Posledično, na linijama svakog kubita prvo se primenjuje H operacija, a zatim sekvenca kontrolisanih faznih operacija ($CP$). Kako se pomeramo nagore kroz kolo, svaki kubit dobija jednu $P$ operaciju više od onog ispod sebe, u skladu sa binarnim ciframa koje kontrolišu akumuliranu fazu.

U oba pristupa, kubiti na kraju budu u **obrnutom poretku** u odnosu na kanonsku Furijeovu bazu. Zato se **na kraj QFT kola obično dodaje niz SWAP operacija** kako bi se povratio ispravan redosled kubita.

<br>
<div align="center">
<img src="images/QFT-3-qubits.png"><br>
<em>Slika: QFT kolo sa 3 kubita (prema udžbeničkom prikazu)</em>
</div>

Inverzna QFT (IQFT) dobija se kada se QFT kolo izvrne, invertuju se sve kontrolisane faze i zamene se kubiti kako bi se povratio njihov originalni raspored.

### Zadatak

- Implementirati funkciju `qft` koja kreira kolo sa n kubita prema udžbeničkom (Nilsen & Čuang) stilu.
- Implementirati inverznu kvantnu Furijeovu transformaciju `iqft`, zasnovanu na prethodnoj funkciji.
- Demonstrirati QFT → IQFT prolazak na najmanje jednom baznom stanju i prikazati rezultate merenja.

### Očekivani rezultat

- Nacrtana kola za QFT i inverzni QFT.  
- Demonstracija QFT → IQFT transformacija na izabranom baznom stanju.  
- Rezultati merenja koji potvrđuje tačnost implementacije.

### Eksperimentisanje

- Testirati funkciju `qft` sa različitim brojem kubita i uporediti dubinu i strukturu kola u udžbeničkom i „odozdo nagore“ pristupu.  
- Primijeniti QFT na različita ulazna stanja (npr. bazna stanja ili superpozicije) i posmatrati raspodele rezultata merenja.  
- Ispitati kako prisustvo ili odsustvo završnih SWAP operacija utiče na redosled izlaznih stanja.  
- Proveriti tačnost `iqft` kombinovanjem QFT → IQFT za različite ulaze i potvrditi povratak na originalno stanje.  
- Implementirati `qft` funkciju koja kreira kolo sa n kubita po **Koppersmitovom (odozdo nagore)** stilu.

---

Reference:

[1] D. Coppersmith, *An approximate Fourier transform useful in quantum factoring*, IBM Research Report RC19642, 1994.

[2] M. A. Nielsen and I. L. Chuang, *Quantum Computation and Quantum Information*, 10th Anniversary Edition, Cambridge Univ. Press, 2010.


In [ ]:
from IPython.display import display

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer
import numpy as np


def qft(n, insert_barriers = True):
    """
    Quantum Fourier Transform (QFT) textbook style:target goes top -> bottom

    Args:
        n: number of qubits
        insert_barriers: keep textbook layering in the drawing
    """
    qc = QuantumCircuit(n)
    for j in range(n):
        qc.h(j)
        # controls are below the target: j+1..n-1
        for k in range(j + 1, n):
            theta = np.pi / (2 ** (k - j))  # π/2, π/4, ...
            qc.cp(theta, k, j)
        if insert_barriers:
            qc.barrier()
    
    for i in range(n // 2):
        qc.swap(i, n - 1 - i)
        
    return qc

def iqft(n, insert_barriers = True):
    """Inverse QFT in the same convention."""
    return qft(n, insert_barriers=insert_barriers).inverse()

# -----------------------------------------------
#                main program
# -----------------------------------------------

n = 4
print("Textbook QFT layout:")
qc_t = qft(n)
display(circuit_drawer(qc_t, output="mpl"))

print("Textbook Inverse QFT layout:")
iqc_t = iqft(n)
display(circuit_drawer(iqc_t, output="mpl"))

# -- test the circuits --
bitstring = "1101" # basis state to test

qc = QuantumCircuit(n)
for i, bit in enumerate(reversed(bitstring)): # prepare the input state
    if bit == "1":
        qc.x(i)   # flip qubit i to |1>
                
qc.compose(qc_t, inplace=True)
qc.compose(iqc_t, inplace=True)
qc.measure_all()

# --- Simulate and show results ---
simulator = AerSimulator()
result = simulator.run(qc, shots=2048).result()
counts = result.get_counts()

print(f"Testing round-trip on |{bitstring}⟩ with exact QFT → iQFT: {counts}")
